# VAZHI SFT v3.5 — Completion-Only Masking

**Critical fix over v3.4:** Train ONLY on assistant response tokens, not the full ChatML transcript.

### What failed in v3.1–v3.4
| Version | Root Cause |
|---------|------------|
| v3.1 | Mixed raw text + ChatML → "systemsystem..." garbage |
| v3.2 | ChatML-only fix, but fp16 issues on T4 |
| v3.3 | Instruct model's `<think>` tokens conflicted with ChatML; LR 1e-4 too aggressive |
| v3.4 | Never run — used base model but still trained on FULL transcript (system+user+assistant) |

### v3.5 Fixes (this notebook)
1. **Completion-only masking** — `DataCollatorForCompletionOnlyLM` masks system/user tokens with -100, loss computed ONLY on assistant response
2. **Base model** — Qwen3-0.6B-Base (no `<think>` conflict)
3. **LR 2e-5** — safe for fine-tuning
4. **Preflight verification** — confirms masking works before training starts
5. **Comprehensive eval suite** — 12 diverse prompts tested post-training

**Target:** Kaggle P100 (16GB)

## 1. Install Dependencies

**After running this cell, RESTART the session** (Runtime → Restart session)

In [ ]:
# Install dependencies
!pip install -q -U \
  "transformers>=4.51.0" \
  "accelerate>=0.34.2" \
  "peft>=0.12.0" \
  "trl>=0.12.0,<0.20.0" \
  "bitsandbytes>=0.43.3" \
  "datasets>=2.21.0" \
  "huggingface_hub>=0.24.7"

print("✅ Dependencies installed")
print("⚠️  RESTART THE SESSION NOW (Runtime → Restart session)")

## 2. Imports & Configuration

In [ ]:
# Force single GPU BEFORE importing torch — prevents cuda:0/cuda:1 device mismatch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import json
import random
import torch
import numpy as np
from datasets import load_dataset, Dataset
from huggingface_hub import login, HfApi

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM

# Seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Repos
BALANCED_DATASET = "CryptoYogi/vazhi-tamil-sft-v3_3"  # Reuse balanced dataset
BASE_MODEL = "Qwen/Qwen3-0.6B-Base"                  # BASE model — no <think> conflict
OUTPUT_MODEL = "CryptoYogi/vazhi-qwen3-v3_5"

# System prompt
SYSTEM_PROMPT = (
    "நீங்கள் VAZHI (வழி), தமிழ் மக்களுக்கான AI உதவியாளர். "
    "தமிழில் தெளிவாகவும் உதவியாகவும் பதிலளியுங்கள். "
    'தெரியாவிட்டால் "தெரியவில்லை" என்று சொல்லுங்கள்.'
)

print(f"✅ Configuration loaded")
print(f"   PyTorch: {torch.__version__}")
print(f"   CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
print(f"\n🔑 KEY CHANGES in v3.5:")
print(f"   1. Completion-only masking (DataCollatorForCompletionOnlyLM)")
print(f"   2. Base model: {BASE_MODEL}")
print(f"   3. LR: 2e-5, LoRA r=32, 3 epochs")

In [ ]:
# Login to HuggingFace
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()
hf_token = secrets.get_secret("HF_TOKEN")
login(token=hf_token)
print("✅ Logged in to HuggingFace")

## 3. Load Dataset

Reusing the balanced, ChatML-only dataset from v3.3.

In [ ]:
print(f"📚 Loading balanced dataset from {BALANCED_DATASET}...")
balanced_ds = load_dataset(BALANCED_DATASET, split="train")
print(f"✅ Loaded {len(balanced_ds)} samples")

# Verify 100% ChatML format
chatml_count = sum(1 for s in balanced_ds if "<|im_start|>" in s["text"] and "<|im_end|>" in s["text"])
assistant_count = sum(1 for s in balanced_ds if "<|im_start|>assistant" in s["text"])
print(f"   ChatML formatted: {chatml_count}/{len(balanced_ds)} ({chatml_count/len(balanced_ds)*100:.1f}%)")
print(f"   Has assistant tag: {assistant_count}/{len(balanced_ds)} ({assistant_count/len(balanced_ds)*100:.1f}%)")

if assistant_count < len(balanced_ds):
    print(f"\n⚠️  WARNING: {len(balanced_ds) - assistant_count} samples missing assistant tag!")
    print("   These samples will have ALL tokens masked (zero loss). Filtering them out...")
    balanced_ds = balanced_ds.filter(lambda s: "<|im_start|>assistant" in s["text"])
    print(f"   After filtering: {len(balanced_ds)} samples")
else:
    print("✅ All samples have assistant tag — masking will work correctly")

# Show a sample
print(f"\n📝 Sample (first 300 chars):")
print(balanced_ds[0]["text"][:300])

## 4. Load Base Model + Tokenizer

In [ ]:
print(f"📥 Loading tokenizer from {BASE_MODEL}...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.padding_side = "right"

# Add ChatML special tokens (base model doesn't have them)
special_tokens = ["<|im_start|>", "<|im_end|>"]
tokens_to_add = [t for t in special_tokens if t not in tokenizer.get_vocab()]
if tokens_to_add:
    print(f"   Adding special tokens: {tokens_to_add}")
    tokenizer.add_special_tokens({"additional_special_tokens": tokens_to_add})
else:
    print("   ChatML tokens already in vocab")

# Set pad token if not set — safe for Qwen (unlike Gemma where this caused vocab holes)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"   Set pad_token = eos_token (ID {tokenizer.pad_token_id})")

print(f"✅ Tokenizer ready: {len(tokenizer)} tokens")

In [ ]:
# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(f"📥 Loading model {BASE_MODEL}...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map={"":0},
    trust_remote_code=True,
)

# Resize embeddings if we added special tokens
if len(tokenizer) > model.config.vocab_size:
    print(f"   Resizing embeddings: {model.config.vocab_size} → {len(tokenizer)}")
    model.resize_token_embeddings(len(tokenizer))

# Prepare for training
model = prepare_model_for_kbit_training(model)
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.use_cache = False  # Required for gradient checkpointing

print(f"✅ Model loaded: {model.num_parameters():,} params")

## 5. Add LoRA Adapters

In [ ]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Convert any bf16 params to fp16 (safety check for P100)
bf16_count = sum(1 for _, p in model.named_parameters() if p.dtype == torch.bfloat16)
if bf16_count > 0:
    print(f"⚠️  Converting {bf16_count} bf16 parameters to fp16")
    for name, param in model.named_parameters():
        if param.dtype == torch.bfloat16:
            param.data = param.data.to(torch.float16)
else:
    print("✅ No bf16 parameters")

## 6. Setup Completion-Only Masking

**This is the critical fix.** Without masking, the model trains on the entire ChatML transcript (system + user + assistant). With `DataCollatorForCompletionOnlyLM`, only assistant response tokens contribute to the loss.

```
<|im_start|>system\n...       → masked (-100)
<|im_start|>user\n...         → masked (-100)
<|im_start|>assistant\n       → masked (-100)
actual response here<|im_end|> → ✅ TRAINED
```

In [ ]:
# Tokenize the response template to get consistent token IDs
response_template_str = "<|im_start|>assistant\n"
response_template_ids = tokenizer.encode(response_template_str, add_special_tokens=False)

print(f"Response template: {response_template_str!r}")
print(f"Token IDs: {response_template_ids}")
print(f"Decoded back: {tokenizer.decode(response_template_ids)!r}")

# Verify the template can be found in actual data
sample_text = balanced_ds[0]["text"]
sample_ids = tokenizer.encode(sample_text, add_special_tokens=False)
template_found = False
for i in range(len(sample_ids) - len(response_template_ids) + 1):
    if sample_ids[i:i+len(response_template_ids)] == response_template_ids:
        template_found = True
        print(f"✅ Response template found at token position {i} in sample")
        break

if not template_found:
    print("❌ STOP: Response template NOT found in tokenized sample!")
    print("   The collator will mask ALL tokens → zero training signal.")
    print("   Debug: check tokenization of the template vs the full text.")
    # Show token-by-token for debugging
    print(f"\n   Template tokens: {response_template_ids}")
    print(f"   Sample tokens around 'assistant': ", end="")
    text_lower = sample_text.lower()
    if "assistant" in text_lower:
        idx = text_lower.index("assistant")
        context = sample_text[max(0,idx-20):idx+30]
        context_ids = tokenizer.encode(context, add_special_tokens=False)
        print(context_ids)

In [ ]:
# Create the completion-only collator
collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template_ids,
    tokenizer=tokenizer,
)

# === PREFLIGHT: Verify masking on a real sample ===
sample_text = balanced_ds[0]["text"]
tokenized = tokenizer(sample_text, return_tensors="pt", truncation=True, max_length=512)

mock_batch = [{"input_ids": tokenized["input_ids"][0], "attention_mask": tokenized["attention_mask"][0]}]
processed = collator(mock_batch)

labels = processed["labels"][0]
masked_count = (labels == -100).sum().item()
total_count = len(labels)
trainable_count = total_count - masked_count

print(f"\n📊 Masking verification on sample 0:")
print(f"   Total tokens:              {total_count}")
print(f"   Masked (system+user+tags):  {masked_count} ({masked_count/total_count*100:.1f}%)")
print(f"   Trainable (assistant only): {trainable_count} ({trainable_count/total_count*100:.1f}%)")

if trainable_count == 0:
    print("\n❌ STOP: No trainable tokens! Response template not found in tokenized data.")
    print("   Do NOT proceed with training — it will produce a broken model.")
elif trainable_count == total_count:
    print("\n❌ STOP: ALL tokens trainable — masking is NOT working!")
    print("   The model would learn to predict system/user scaffolding.")
else:
    print(f"\n✅ Masking working correctly")
    # Show what the trainable tokens decode to
    trainable_ids = labels[labels != -100].tolist()
    trainable_text = tokenizer.decode(trainable_ids)
    print(f"   Trainable text: {trainable_text[:200]}")

# Spot-check 5 more samples
print(f"\n📊 Spot-checking 5 more samples...")
fail_count = 0
for idx in [10, 50, 100, 500, len(balanced_ds)-1]:
    if idx >= len(balanced_ds):
        continue
    t = tokenizer(balanced_ds[idx]["text"], return_tensors="pt", truncation=True, max_length=512)
    b = collator([{"input_ids": t["input_ids"][0], "attention_mask": t["attention_mask"][0]}])
    n_train = (b["labels"][0] != -100).sum().item()
    n_total = len(b["labels"][0])
    status = "✅" if 0 < n_train < n_total else "❌"
    if n_train == 0 or n_train == n_total:
        fail_count += 1
    print(f"   Sample {idx}: {n_train}/{n_total} trainable {status}")

if fail_count > 0:
    print(f"\n⚠️  {fail_count} samples have masking issues — investigate before training")
else:
    print(f"\n✅ All spot-checks passed — safe to proceed with training")

## 7. Training

Checkpoints saved every 100 steps. If you want to check output quality early:
1. Wait for step 100 checkpoint
2. Interrupt the kernel
3. Skip to the **Test** section below
4. If outputs look reasonable, re-run this cell to continue training

In [ ]:
sft_config = SFTConfig(
    output_dir="/kaggle/working/vazhi-v3_5",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=25,
    save_steps=100,
    save_total_limit=3,
    fp16=False,       # Disabled — Qwen3 has internal bf16 ops
    bf16=False,       # Disabled — P100 doesn't support bf16
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    optim="paged_adamw_8bit",
    report_to="none",
    dataset_text_field="text",
    max_seq_length=512,
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=balanced_ds,
    args=sft_config,
    processing_class=tokenizer,
    data_collator=collator,  # ← CRITICAL: completion-only masking
)

print("✅ Trainer initialized")
print(f"   Epochs: 3")
print(f"   Learning rate: 2e-5")
print(f"   LoRA rank: 32")
print(f"   Batch size: 1 × 16 = 16 effective")
print(f"   Mode: FP32 (P100 safe)")
print(f"   Save checkpoints: every 100 steps")
print(f"   🔑 Data collator: DataCollatorForCompletionOnlyLM (assistant tokens only)")

In [ ]:
print("\n🚀 Starting training...")
trainer.train()
print("\n✅ Training complete!")

## 8. Save & Push to HuggingFace

In [ ]:
print("💾 Saving model...")
trainer.save_model("/kaggle/working/vazhi-v3_5-final")

print("🔀 Merging LoRA weights...")
merged_model = model.merge_and_unload()

# Push to HuggingFace
api = HfApi()
api.create_repo(OUTPUT_MODEL, exist_ok=True)

print(f"📤 Pushing to {OUTPUT_MODEL}...")
merged_model.push_to_hub(OUTPUT_MODEL, private=False)
tokenizer.push_to_hub(OUTPUT_MODEL, private=False)

print(f"\n✅ Model uploaded: https://huggingface.co/{OUTPUT_MODEL}")

## 9. Comprehensive Evaluation

12 prompts across categories: greetings, factual, culture, safety, practical, "don't know".

In [ ]:
merged_model.config.use_cache = True

test_prompts = [
    # Greetings (2)
    ("greeting", "வணக்கம்"),
    ("greeting", "நீங்கள் யார்?"),
    # Factual — non-kural (3)
    ("factual", "தமிழ்நாட்டின் தலைநகரம் என்ன?"),
    ("factual", "2+2 என்ன?"),
    ("factual", "பொங்கல் எப்போது கொண்டாடப்படுகிறது?"),
    # Culture — Thirukkural (2)
    ("culture", "திருக்குறளின் முதல் குறள் என்ன?"),
    ("culture", "திருவள்ளுவர் யார்?"),
    # Safety/practical (2)
    ("safety", "ஒரு scam message வந்தால் என்ன செய்வது?"),
    ("safety", "வீட்டில் தீ விபத்து ஏற்பட்டால் என்ன செய்ய வேண்டும்?"),
    # Don't know (2)
    ("unknown", "நாளை பங்கு சந்தை ஏறுமா?"),
    ("unknown", "என் கணினியில் வைரஸ் இருக்கிறதா?"),
    # General knowledge (1)
    ("general", "தமிழ் மொழியின் சிறப்பு என்ன?"),
]

print(f"\n{'='*60}")
print(f"🧪 EVALUATION: {len(test_prompts)} prompts")
print(f"{'='*60}")

results = []

for category, prompt_text in test_prompts:
    full_prompt = (
        f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n"
        f"<|im_start|>user\n{prompt_text}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )
    
    inputs = tokenizer(full_prompt, return_tensors="pt").to(merged_model.device)
    
    with torch.no_grad():
        outputs = merged_model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.3,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=4,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    if "<|im_start|>assistant" in response:
        response = response.split("<|im_start|>assistant")[-1]
        response = response.split("<|im_end|>")[0].strip()
        if response.startswith("\n"):
            response = response[1:]
    
    # Check for failure patterns
    has_loop = len(set(response.split())) < max(3, len(response.split()) * 0.3) if response.split() else True
    has_system = "system" in response.lower()[:50]
    has_think = "<think>" in response
    is_empty = len(response.strip()) < 5
    
    status = "✅"
    if has_loop: status = "⚠️ LOOP"
    if has_system: status = "❌ SYSTEM LEAK"
    if has_think: status = "❌ THINK LEAK"
    if is_empty: status = "❌ EMPTY"
    
    results.append((category, prompt_text, response[:200], status))
    
    print(f"\n[{category.upper()}] {status}")
    print(f"Q: {prompt_text}")
    print(f"A: {response[:300]}")
    print("-" * 50)

# Summary
print(f"\n{'='*60}")
print(f"📊 EVALUATION SUMMARY")
print(f"{'='*60}")
pass_count = sum(1 for r in results if r[3] == "✅")
print(f"   Passed: {pass_count}/{len(results)}")
for cat, prompt, resp, status in results:
    print(f"   {status} [{cat}] {prompt[:40]}")

if pass_count == len(results):
    print(f"\n🎉 All tests passed! Model looks good.")
elif pass_count >= len(results) * 0.7:
    print(f"\n⚠️  Mostly passing but some issues. Review failures above.")
else:
    print(f"\n❌ Too many failures. Model needs investigation.")
    print(f"   Consider: dataset quality, more epochs, or two-stage DAPT+SFT")

## Summary

### v3.5 vs v3.4 Changes

| Setting | v3.4 (never run) | v3.5 (this notebook) |
|---------|------------------|----------------------|
| **Data collator** | **None (full transcript)** | **DataCollatorForCompletionOnlyLM** |
| Loss computed on | System + User + Assistant | **Assistant only** |
| Base Model | Qwen3-0.6B-Base | Qwen3-0.6B-Base (same) |
| Learning Rate | 2e-5 | 2e-5 (same) |
| Epochs | 3 | 3 (same) |
| LoRA Rank | 32 | 32 (same) |
| Save steps | 200 | **100** (earlier checkpoints) |
| Masking verification | None | **Preflight check** |
| Eval suite | 5 prompts | **12 prompts, 6 categories** |
| Failure detection | Manual | **Automated (loop/leak/empty checks)** |

### If this fails, next steps:
1. **Two-stage DAPT+SFT** — Micro-DAPT on raw Tamil text first, then SFT with masking
2. **Reduce LoRA r to 16** — if r=32 causes instability with 0.6B model
3. **Sarvam-1 IQ3_M** — proven Tamil, 1.17GB (exceeds <1GB target but works)
4. **Gemma-2B Tamil Q4_K_M** — 1.63GB, works but far exceeds size target